# Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter
import json
import math
import statistics

import seaborn
from sklearn import preprocessing
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import tensorflow as tf
from sklearn.impute import KNNImputer

# Upload Datasets

In [2]:
Data_File1 = pd.read_csv (r'/homedir01/eeldeeb19/Oulu Data/sc-export-2021022500-elsys.csv')

In [3]:
Data_File1.head()

,time,deveui,temperature,humidity,light,motion,co2,battery,sound_avg,sound_peak,moisture,pressure,acceleration_x,acceleration_y,acceleration_z
0,1596240002191,a8-17-58-ff-fe-04-64-6b,21.4,44.0,0.0,79.0,NaN,3.635,34.0,64.0,NaN,NaN,NaN,NaN,NaN
1,1596240006323,a8-17-58-ff-fe-04-63-af,21.5,46.0,0.0,86.0,NaN,3.638,34.0,64.0,NaN,NaN,NaN,NaN,NaN
2,1596240010305,a8-17-58-ff-fe-04-63-b0,21.5,47.0,0.0,76.0,NaN,3.638,34.0,64.0,NaN,NaN,NaN,NaN,NaN
3,1596240012968,a8-17-58-ff-fe-03-0f-71,21.5,45.0,0.0,5.0,500.0,3.684,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1596240013257,a8-17-58-ff-fe-03-10-45,21.5,47.0,0.0,5.0,479.0,3.686,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Separate the devices

In [4]:
Time_1 = Data_File1["time"]

Deveui_1 = Data_File1["deveui"]
Devices_1 = dict(Counter(Deveui_1))
print(len(Devices_1))
# print(a)

Devices_Headers_1 = list(Devices_1.keys())
# print(Devices_Headers_1)
Deveui_lis_1 = list(Deveui_1)

468


In [5]:
all_list_1 = []
for i in range (0,len(Devices_Headers_1)) :
    res_list_1 = []
    for j in range (0,len(Deveui_lis_1)) :
        if Deveui_lis_1[j] == Devices_Headers_1[i] : 
            res_list_1.append(j)
    all_list_1.append(res_list_1)
#print(Deveui_lis_1.index(it[256]))
print(all_list_1)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
Co2 = Data_File1["co2"]
count = 0

for i in range (0,len(Devices_Headers_1)):
    t = all_list_1[i]
    Co2_i = Co2[t]
    Co2_i = [kk for kk in Co2_i if str(kk) != 'nan']
    if len(Co2_i) == 0:
        count = count + 1
    else:
        id_dev = str(Devices_Headers_1[i])
        df2 = pd.read_csv('New folder/Dev_' + id_dev + '_new.csv')
        df = df2[['Co2']].copy()
        df[df > 10000] = 0
        df[df < 50] = 0
        df_failure = df.replace(0, np.nan) # Replace missing value with nan

        # Linear Interpolation
        df_linear = df_failure.interpolate()
        df_linear.to_csv('Interpolated/Dev_' + Devices_Headers_1[i] + '_Linear.csv', index=True, header=True, sep=',')        
        
        # KNN
        imputer = KNNImputer(n_neighbors=7)
        imputed = imputer.fit_transform(df_failure)
        df_K = pd.DataFrame(imputed, columns=df_failure.columns)
        df_K.to_csv('Interpolated/Dev_' + Devices_Headers_1[i] + '_K.csv', index=True, header=True, sep=',')
        
        

We use linear interpolation and KNN interpolation to handle the missing values as indicated in teh notebook 4. Test_Interpolation.ipynb

Here, we save the linear intepolation technique as csv file with a name of the ID of the device _Linear, whereas we name the KNN interpolation as the ID of the device _K. We use K = 7 for the KNN interpolation.